In [2]:
import numpy as np
import json

In [34]:
filepath = 'users.json'
username = 'Andrew'

cards_filepath = 'data/card_data.json'
card = 'Chase Sapphire Preferred'

with open(filepath, 'r') as f:
    users = json.load(f)

with open(cards_filepath, 'r') as f:
    cards = json.load(f)

andrew = users[username]
andrew_spend = andrew['spending_categories']
andrew


{'credit_score': '740',
 'credit_age': '2',
 'spending_categories': {'monthly_dining': 600,
  'monthly_groceries': 100,
  'monthly_gas': 0,
  'monthly_streaming': 7,
  'monthly_online_shopping': 12,
  'monthly_other': 200,
  'monthly_travel': 10,
  'monthly_hotel': 5,
  'monthly_hotel_portal': 6,
  'monthly_flights': 7,
  'monthly_flights_portal': 8,
  'monthly_car_rental': 9,
  'monthly_car_rental_portal': 10}}

In [35]:
csp = cards[card]
csp

{'issuer': 'Chase',
 'annual_fee': 95,
 'purpose': 'Travel',
 'good_airlines': ['Southwest', 'United'],
 'spending_categories': {'Chase Travel': 5,
  'Dining': 3,
  'Online_Grocery': 3,
  'Streaming': 3,
  'Travel': 2,
  'Other': 1}}

In [36]:
MASTER = {
    'monthly_dining': 'Dining',
    'monthly_groceries': 'Grocery',
    'monthly_gas': 'Gas',
    'monthly_streaming': 'Streaming',
    'monthly_online_shopping': 'Online_Shopping',
    'monthly_other': 'Other',

    # travel (non-portal)
    'monthly_travel': 'Travel',
    'monthly_hotel': 'Hotel',
    'monthly_flights': 'Flights',

    # travel (portal)
    'monthly_hotel_portal': 'PORTAL_HOTEL',
    'monthly_flights_portal': 'PORTAL_FLIGHT',
}

PORTAL_BY_ISSUER = {
    'American Express': {
        'unified': 'AmexTravel',
        'hotel':  'AmexTravel Hotels (Prepaid)',
        'flight': 'AmexTravel Flights',
    },
    'Chase': {
        'unified': 'Chase Travel',
        'hotel':  'Chase Travel - Hotels',
        'flight': 'Chase Travel - Flights',
    },
    'Capital One': {
        'unified': 'Capital One Travel',
        'hotel':  'Capital One Travel - Hotels',
        'flight': 'Capital One Travel - Flights',
    },
}


def resolve_formal(formal_cat: str, card: dict) -> str:
    
    # If the category is not related to a portal category, return the category (e.g. Dining, Grocery, etc.)
    if formal_cat not in ('PORTAL_HOTEL', 'PORTAL_FLIGHT'):
        return formal_cat

    issuer = card.get('issuer')

    # Single out the travel portal for the issuer
    portal = PORTAL_BY_ISSUER.get(issuer, {})
    card_cats = card.get('spending_categories', {})

    # If the card has a specific label (e.g. Chase Travel - Hotel) use that 
    if formal_cat == 'PORTAL_HOTEL':
        specific = portal.get('hotel')
        unified  = portal.get('unified')
        # If the card has a specific label (e.g. Chase Travel - Hotel) use that 
        if specific and specific in card_cats:
            return specific
        # Not the card doesn't have portal cateogry for it, fallback to general travel portal
        if unified and unified in card_cats:
            return unified
        return 'Other'  # last-resort fallback

    if formal_cat == 'PORTAL_FLIGHT':
        specific = portal.get('flight')
        unified  = portal.get('unified')
        if specific and specific in card_cats:
            return specific
        if unified and unified in card_cats:
            return unified
        return 'Other'


def calc_point_return(user, card):
    user_spend = user['spending_categories']
    card_cats  = card['spending_categories']

    total_points = 0
    breakdown = {}

    for user_cat, amt in user_spend.items():
        # 1) default unknown user categories -> 'Other'
        formal = resolve_formal(MASTER.get(user_cat, 'Other'), card)
        # 2) if the card lacks that formal category, fall back to card's 'Other' (or 1x)
        if formal in card_cats:
            earn_rate = card_cats[formal]
        elif 'Other' in card_cats:
            earn_rate = card_cats['Other']
        else:
            earn_rate = 1

        pts = amt * earn_rate

        total_points += pts
        breakdown[user_cat] = dict(amount=amt, formal=formal,
                                   earn_rate=earn_rate, points=pts)

    return total_points, breakdown


In [44]:
calc_point_return(andrew, csp)[1]['monthly_hotel']

{'amount': 5, 'formal': 'Hotel', 'earn_rate': 1, 'points': 5}

In [38]:
# calculate 

In [9]:
# benefit_utilization_rate

# approval odds metric